In [1]:
import polars as pl

In [2]:
contig_otus = pl.read_csv('/home/n10927662/rossenzhao/land-use-viromes/lyrebird-0.3.1/lyrebird_0.3.1_contigs.tsv', separator='\t')
print(contig_otus.shape)

(350470, 12)


In [3]:
high_quality_mags = set()
with open("/home/n10927662/rossenzhao/land-use-viromes/hq-complete-ids.txt") as f:
    for line in f:
        high_quality_mags.add(line.strip())

## only root and d__Viruses

In [4]:
### number of genes per contig
contig_to_genes = {}
for line in contig_otus.iter_rows():
    gene = line[0]
    read_names = line[6]
    taxonomy = line[5]
    if taxonomy == "Root" or "d__Viruses" in taxonomy:
        for read_name in read_names.split():
            contig = read_name.rsplit('_', 3)[0]
            # if contig not in high_quality_mags:
            #     continue
            if contig not in contig_to_genes:
                contig_to_genes[contig] = set()
            contig_to_genes[contig].add(gene)

avg_genes_per_contig = sum(len(genes) for genes in contig_to_genes.values()) / len(contig_to_genes)
avg_genes_per_contig

4.522361813600188

In [5]:
# all_otus = pl.read_csv('/work/microbiome/msingle/rossenzhao/land-use-viromes/lyrebird-0.3.1/otu_tables/*.tsv', separator='\t')
all_otus = pl.read_csv('/work/microbiome/msingle/rossenzhao/land-use-viromes/lyrebird-0.3.1/lyrebird_0.3.1_clustered_otu_table.tsv', separator='\t')
print(all_otus.shape)
# drop read_unaligned_sequences and equal_best_hit_taxonomies column
# all_otus = all_otus.drop(['read_unaligned_sequences', 'equal_best_hit_taxonomies'])
root_otus = all_otus.filter(pl.col('taxonomy') == "Root")
print(root_otus.shape)
viral_otus = all_otus.filter(pl.col('taxonomy').str.contains("d__Viruses"))
viral_otus = pl.concat([viral_otus, root_otus])
viral_otus.head()
del all_otus, root_otus
print(viral_otus.shape)

(7011014, 6)
(628571, 6)
(4704250, 6)


In [6]:
# join to contigs via contig OTU table
contig_otus_join = viral_otus.join(contig_otus, on='sequence', how='inner')
print(contig_otus_join.shape)
contig_otus_join.head()

(335359, 17)


gene,sample,sequence,num_hits,coverage,taxonomy,gene_right,sample_right,num_hits_right,coverage_right,taxonomy_right,read_names,nucleotides_aligned,taxonomy_by_known?,read_unaligned_sequences,equal_best_hit_taxonomies,taxonomy_assignment_method
str,str,str,i64,f64,str,str,str,i64,f64,str,str,str,bool,str,str,str
"""phrog_5302""","""SRR14194036_1""","""ACCGCCGCCCGCAACGCCTGTAGTGCAAAA…",23130,38468.8,"""Root; d__Viruses; p__Urovirico…","""phrog_5302""","""contigs2""",1,1.0,"""Root; d__Viruses; p__Urovirico…","""CG12_NODE_119_length_42942_cov…","""60""",false,"""CTGACCTCGCCGAAGCCGGGACCGGCTCGC…","""['phrog_5302~IMGVR_UViG_330000…","""diamond"""
"""phrog_5302""","""SRR14194039_1""","""ACCGCCGCCCGCAACGCCTGTAGTGCAAAA…",8759,14567.59,"""Root; d__Viruses; p__Urovirico…","""phrog_5302""","""contigs2""",1,1.0,"""Root; d__Viruses; p__Urovirico…","""CG12_NODE_119_length_42942_cov…","""60""",false,"""CTGACCTCGCCGAAGCCGGGACCGGCTCGC…","""['phrog_5302~IMGVR_UViG_330000…","""diamond"""
"""phrog_5302""","""SRR14194055_1""","""ACCGCCGCCCGCAACGCCTGTAGTGCAAAA…",12,19.96,"""Root; d__Viruses; p__Urovirico…","""phrog_5302""","""contigs2""",1,1.0,"""Root; d__Viruses; p__Urovirico…","""CG12_NODE_119_length_42942_cov…","""60""",false,"""CTGACCTCGCCGAAGCCGGGACCGGCTCGC…","""['phrog_5302~IMGVR_UViG_330000…","""diamond"""
"""phrog_5302""","""SRR14194046_1""","""ACCGCCGCCCGCAACGCCTGTAGTGCAAAA…",11,18.29,"""Root; d__Viruses; p__Urovirico…","""phrog_5302""","""contigs2""",1,1.0,"""Root; d__Viruses; p__Urovirico…","""CG12_NODE_119_length_42942_cov…","""60""",false,"""CTGACCTCGCCGAAGCCGGGACCGGCTCGC…","""['phrog_5302~IMGVR_UViG_330000…","""diamond"""
"""phrog_5302""","""SRR14194056_1""","""ACCGCCGCCCGCAACGCCTGTAGTGCAAAA…",11,18.29,"""Root; d__Viruses; p__Urovirico…","""phrog_5302""","""contigs2""",1,1.0,"""Root; d__Viruses; p__Urovirico…","""CG12_NODE_119_length_42942_cov…","""60""",false,"""CTGACCTCGCCGAAGCCGGGACCGGCTCGC…","""['phrog_5302~IMGVR_UViG_330000…","""diamond"""


In [7]:
sample1_lyrebird_detected = set() # contig sequences detected by Lyrebird from reads
# for i in contig_otus_join['read_names_right']:
for i in contig_otus_join['read_names']:
    for j in i.split(' '):
        sample1_lyrebird_detected.add(j.rsplit('_', 3)[0])


In [8]:
# contigs detected from contig otus
contig_otus_detected = set() # contig sequences detected by Lyrebird from contigs
for i in contig_otus.filter((pl.col('taxonomy').str.contains("d__Viruses")) | (pl.col('taxonomy') == "Root"))['read_names']:
# for i in contig_otus['read_names']:
    for j in i.split(' '):
        contig_otus_detected.add(j.rsplit('_', 3)[0])
print(len(contig_otus_detected))
print(len(contig_otus_detected)/59626 * 100)

55161
92.51165598899809


### Number detected ###

In [9]:
print(len(sample1_lyrebird_detected))
print(len(sample1_lyrebird_detected) / 59626 * 100)

47153
79.08127327004998


In [10]:
### unique contig window sequences
viral_contig_windows = contig_otus.filter((pl.col('taxonomy').str.contains("d__Viruses")) | (pl.col('taxonomy') == "Root"))
unique_contig_windows = set()
for sequence in viral_contig_windows["sequence"]:
# for sequence in contig_otus_join["sequence"]:
    unique_contig_windows.add(sequence)
print(len(unique_contig_windows))

254370


In [11]:
# unique reads window sequences
unique_reads_windows = set()
# for sequence in viral_otus["sequence"]:
for sequence in viral_otus["sequence"]:
    unique_reads_windows.add(sequence)
print(len(unique_reads_windows))

3473530


In [12]:
num_read_and_contig_windows = len(unique_reads_windows.intersection(unique_contig_windows))
num_only_read_windows = len(unique_reads_windows - unique_contig_windows)
est_num_contig_windows = len(unique_contig_windows) * 59626 / len(sample1_lyrebird_detected)
print("est_num_contig_windows if lyrebird detected all: " + str(est_num_contig_windows)) # estimated number of contig windows if lyrebird detected all contigs
print("reads + contigs windows: " + str(num_read_and_contig_windows)) # number of windows that are both in reads and contigs
print("reads-only windows: " + str(num_only_read_windows)) # number of windows that are only in reads
print("number of lyrebird windows vs contigs: " + str(num_only_read_windows / est_num_contig_windows)) # how many times more windows lyrebird detected vs contigs
print("number of non-contig viruses: " + str(num_only_read_windows / avg_genes_per_contig)) # estimated total number of unique viruses not found in contigs
# print(num_only_read_windows / 10.48039840176038) # estimated total number of unique viruses not found in contigs
lyrebird_unique_estimate = len(unique_reads_windows) / avg_genes_per_contig
# lyrebird_unique_estimate = len(unique_reads_windows) / 10.48039840176038
print("total number of unique lyrebird windows: " + str(lyrebird_unique_estimate)) # estimated total number of unique viruses found by lyrebird
print(lyrebird_unique_estimate/59696)

est_num_contig_windows if lyrebird detected all: 321656.42949547217
reads + contigs windows: 131947
reads-only windows: 3341583
number of lyrebird windows vs contigs: 10.388671556298048
number of non-contig viruses: 506260.9341283608
total number of unique lyrebird windows: 526251.3433073142
8.81552102833212


## include all

In [52]:
### number of genes per contig
contig_to_genes = {}
for line in contig_otus.iter_rows():
    gene = line[0]
    read_names = line[6]
    taxonomy = line[5]
    for read_name in read_names.split():
        contig = read_name.rsplit('_', 3)[0]
        if contig not in high_quality_mags:
            continue
        if contig not in contig_to_genes:
            contig_to_genes[contig] = set()
        contig_to_genes[contig].add(gene)

avg_genes_per_contig = sum(len(genes) for genes in contig_to_genes.values()) / len(contig_to_genes)
avg_genes_per_contig

8.829737114859135

In [40]:
# all_otus = pl.read_csv('/work/microbiome/msingle/rossenzhao/land-use-viromes/lyrebird-0.3.1/otu_tables/*.tsv', separator='\t')
all_otus = pl.read_csv('/work/microbiome/msingle/rossenzhao/land-use-viromes/lyrebird-0.3.1/lyrebird_0.3.1_clustered_otu_table.tsv', separator='\t')
print(all_otus.shape)

(7011014, 6)


In [41]:
# join to contigs via contig OTU table
contig_otus_join = all_otus.join(contig_otus, on='sequence', how='inner')
print(contig_otus_join.shape)
contig_otus_join.head()

(445814, 17)


gene,sample,sequence,num_hits,coverage,taxonomy,gene_right,sample_right,num_hits_right,coverage_right,taxonomy_right,read_names,nucleotides_aligned,taxonomy_by_known?,read_unaligned_sequences,equal_best_hit_taxonomies,taxonomy_assignment_method
str,str,str,i64,f64,str,str,str,i64,f64,str,str,str,bool,str,str,str
"""phrog_5302""","""SRR14194036_1""","""ACCGCCGCCCGCAACGCCTGTAGTGCAAAA…",23130,38468.8,"""Root; d__Viruses; p__Urovirico…","""phrog_5302""","""contigs2""",1,1.0,"""Root; d__Viruses; p__Urovirico…","""CG12_NODE_119_length_42942_cov…","""60""",false,"""CTGACCTCGCCGAAGCCGGGACCGGCTCGC…","""['phrog_5302~IMGVR_UViG_330000…","""diamond"""
"""phrog_5302""","""SRR14194039_1""","""ACCGCCGCCCGCAACGCCTGTAGTGCAAAA…",8759,14567.59,"""Root; d__Viruses; p__Urovirico…","""phrog_5302""","""contigs2""",1,1.0,"""Root; d__Viruses; p__Urovirico…","""CG12_NODE_119_length_42942_cov…","""60""",false,"""CTGACCTCGCCGAAGCCGGGACCGGCTCGC…","""['phrog_5302~IMGVR_UViG_330000…","""diamond"""
"""phrog_5302""","""SRR14194055_1""","""ACCGCCGCCCGCAACGCCTGTAGTGCAAAA…",12,19.96,"""Root; d__Viruses; p__Urovirico…","""phrog_5302""","""contigs2""",1,1.0,"""Root; d__Viruses; p__Urovirico…","""CG12_NODE_119_length_42942_cov…","""60""",false,"""CTGACCTCGCCGAAGCCGGGACCGGCTCGC…","""['phrog_5302~IMGVR_UViG_330000…","""diamond"""
"""phrog_5302""","""SRR14194046_1""","""ACCGCCGCCCGCAACGCCTGTAGTGCAAAA…",11,18.29,"""Root; d__Viruses; p__Urovirico…","""phrog_5302""","""contigs2""",1,1.0,"""Root; d__Viruses; p__Urovirico…","""CG12_NODE_119_length_42942_cov…","""60""",false,"""CTGACCTCGCCGAAGCCGGGACCGGCTCGC…","""['phrog_5302~IMGVR_UViG_330000…","""diamond"""
"""phrog_5302""","""SRR14194056_1""","""ACCGCCGCCCGCAACGCCTGTAGTGCAAAA…",11,18.29,"""Root; d__Viruses; p__Urovirico…","""phrog_5302""","""contigs2""",1,1.0,"""Root; d__Viruses; p__Urovirico…","""CG12_NODE_119_length_42942_cov…","""60""",false,"""CTGACCTCGCCGAAGCCGGGACCGGCTCGC…","""['phrog_5302~IMGVR_UViG_330000…","""diamond"""


In [42]:
sample1_lyrebird_detected = set() # contig sequences detected by Lyrebird from reads
# for i in contig_otus_join['read_names_right']:
for i in contig_otus_join['read_names']:
    for j in i.split(' '):
        sample1_lyrebird_detected.add(j.rsplit('_', 3)[0])

In [ ]:
# contigs detected from contig otus
contig_otus_detected = set() # contig sequences detected by Lyrebird from contigs
# for i in contig_otus.filter((pl.col('taxonomy').str.contains("d__Viruses")) | (pl.col('taxonomy') == "Root"))['read_names']:
for i in contig_otus['read_names']:
    for j in i.split(' '):
        contig_otus_detected.add(j.rsplit('_', 3)[0])
print(len(contig_otus_detected))
print(len(contig_otus_detected)/59626 * 100)

57656
96.69607218327575


### Number detected

In [44]:
print(len(sample1_lyrebird_detected))
print(len(sample1_lyrebird_detected) / 59626 * 100)

52016
87.23711132727334


In [45]:
### unique contig window sequences
# viral_contig_windows = contig_otus.filter((pl.col('taxonomy').str.contains("d__Viruses")) | (pl.col('taxonomy') == "Root"))
unique_contig_windows = set()
# for sequence in viral_contig_windows["sequence"]:
for sequence in contig_otus_join["sequence"]:
    unique_contig_windows.add(sequence)
print(len(unique_contig_windows))

193883


In [48]:
# unique reads window sequences
unique_reads_windows = set()
for sequence in all_otus["sequence"]:
    unique_reads_windows.add(sequence)
print(len(unique_reads_windows))

5230130


In [53]:
num_read_and_contig_windows = len(unique_reads_windows.intersection(unique_contig_windows))
num_only_read_windows = len(unique_reads_windows - unique_contig_windows)
est_num_contig_windows = len(unique_contig_windows) * 59626 / len(sample1_lyrebird_detected)
print("est_num_contig_windows if lyrebird detected all: " + str(est_num_contig_windows)) # estimated number of contig windows if lyrebird detected all contigs
print("reads + contigs windows: " + str(num_read_and_contig_windows)) # number of windows that are both in reads and contigs
print("reads-only windows: " + str(num_only_read_windows)) # number of windows that are only in reads
print("number of lyrebird windows vs contigs: " + str(num_only_read_windows / est_num_contig_windows)) # how many times more windows lyrebird detected vs contigs
print("number of non-contig viruses: " + str(num_only_read_windows / avg_genes_per_contig)) # estimated total number of unique viruses not found in contigs
lyrebird_unique_estimate = len(unique_reads_windows) / avg_genes_per_contig
print("total number of unique lyrebird windows: " + str(lyrebird_unique_estimate)) # estimated total number of unique viruses found by lyrebird
print(lyrebird_unique_estimate/59696)

est_num_contig_windows if lyrebird detected all: 222248.30356044293
reads + contigs windows: 193883
reads-only windows: 5036247
number of lyrebird windows vs contigs: 22.660451932900067
number of non-contig viruses: 570373.3796926689
total number of unique lyrebird windows: 592331.3380642408
9.922462779151715


## more

In [23]:
covs = 0
for cov in viral_otus["coverage"]:
    covs += float(cov)

print(covs)
print(covs/avg_genes_per_contig)
print(covs/avg_genes_per_contig * 20000)

36697875.337290645
5559850.121568997
111197002431.37994


In [24]:
total_covs = 0
for cov in all_otus["coverage"]:
    total_covs += float(cov)
print(total_covs)

52820504.7455288


In [25]:
fastq_counts = "/home/n10927662/rossenzhao/land-use-viromes/fq_count.txt"
basecount=0
for line in open(fastq_counts):
    bases = int(line.strip().split('\t')[-1])
    basecount += bases
print(basecount)

949057697477


In [26]:
(covs/avg_genes_per_contig * 50000)/basecount

0.29291423147135576

In [13]:
with open("../lyrebird_vs_contigs3.csv", "w+") as f:
    f.write("category,count\n")
    f.write("OTUs only in reads," + str(num_only_read_windows) + "\n")
    f.write("OTUs in contigs," + str(len(unique_contig_windows)) + "\n")

with open("../contig_with_otu_vs_not3.csv", "w+") as f:
    f.write("category,count\n")
    f.write("Contigs without OTUs in reads," + str(59696-len(sample1_lyrebird_detected)) + "\n")
    f.write("Contigs with OTUs in reads," + str(len(sample1_lyrebird_detected)) + "\n")

In [14]:
phanta_results = "/home/n10927662/rossenzhao/land-use-viromes/phanta/outputs/final_merged_outputs/relative_taxonomic_abundance.txt"
phanta_viral_count = 0
with open(phanta_results) as f:
    for line in f:
        if "superkingdom_Viruses" in line:
            phanta_viral_count += 1
print(phanta_viral_count)

import os
metaphlan4_folder = "/home/n10927662/rossenzhao/land-use-viromes/metaphlan/prof"
metaphlan_viral_count = 0
for filename in os.listdir(metaphlan4_folder):
    if "vsc.txt" in filename:
        with open(os.path.join(metaphlan4_folder, filename)) as f:
            metaphlan_viral_count += len(f.readlines())-4
print(metaphlan_viral_count)

with open("../tool_results3.csv", "w+") as f:
    f.write("tool,count\n")
    f.write("lyrebird," + str(int(round(lyrebird_unique_estimate))) + "\n")
    f.write("phanta," + str(phanta_viral_count) + "\n")
    f.write("metaphlan4," + str(metaphlan_viral_count) + "\n")

162
37


In [18]:
with open("../genes_per_contig3.csv", "w+") as w:
    w.write("genes_per_contig\n")
    for contig in contig_to_genes:
        w.write(f"{len(contig_to_genes[contig])}\n")